In [1]:
# test if the last h represents sentence meaning

In [3]:
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import dictionary_corpus
from dictionary_corpus import Corpus
import numpy as np
from sklearn.decomposition import PCA, SparsePCA, KernelPCA, IncrementalPCA
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 30)

matplotlib.use('webagg')
# load the model and the corpus
model = torch.load('hidden650_batch128_dropout0.2_lr20.0.pt',map_location=torch.device('cpu'))
corpus = Corpus('')
# print("Vocab size %d", ntokens)

In [194]:
def sent_feature_extraction(sent,file_type,model=model, corpus=corpus): 
    with torch.no_grad():
        words = sent.split()
        target_word = words[-1]
        tokenized_sent=[]
        for word in words:
            if word in corpus.dictionary.word2idx.keys():
                wordidx = corpus.dictionary.word2idx[word]
            else:
                print(f'{word} is to be deleted')
                wordidx = corpus.dictionary.word2idx['<unk>']
            tokenized_sent.append(wordidx)
        # initailize the hidden state of the model
        hidden = model.init_hidden(1)
        # for printing
        print(sent, '\ntokenized as',tokenized_sent)
        # iterate through the whole sentence
        for i, wordid in enumerate(tokenized_sent):
            word = corpus.dictionary.idx2word[wordid]
            # we are not gonna use the output
            _,hidden,_= model(torch.as_tensor(wordid).reshape(1,1), hidden)
            # the hidden embedding is the embedding we wanted.
            hidden_embedding = hidden[0][1].view(650).numpy()
            if word == target_word:
                df = {'label':words[i-1]+' '+target_word+file_type,'tensor':hidden_embedding}
                break
        return df

In [195]:
sents = []
# sents.append('Mary loves John')
sents.append('in the garden , i gave here a card')
sents.append('I gave her a card in the garden')
sents.append('She was given a card by me in the garden')
sents.append('I was given a card by her in the garden')
sents.append('In the garden, she gave me a card')
sents.append('She gave me a card in the garden')
# sents.append('John loves Mary')
# sents.append('John loves Mary so much .')
# sents.append('Mary likes John')
# sents.append('John likes Mary')
sents.append('Mary hates John')
sents.append('John hates Mary')
sents.append('Mary does not like John')
sents.append('John does not like Mary')
# sents.append('Mary is in love with John')
# sents.append('John is in love with Mary')

In [196]:
x = pd.DataFrame(columns = ['tensor'])
for i,each in enumerate(sents):
    if i%2==0:
        x=x.append(sent_feature_extraction(each,str(i//2)),ignore_index=True)
    else:
        x=x.append(sent_feature_extraction(each,str(i//2)),ignore_index=True)

in the garden , i gave here a card 
tokenized as [29, 3, 9293, 11, 17801, 2186, 2920, 42, 748]
I gave her a card in the garden 
tokenized as [162, 2186, 199, 42, 748, 29, 3, 9293]
She was given a card by me in the garden 
tokenized as [0, 138, 915, 42, 748, 155, 4333, 29, 3, 9293]
I was given a card by her in the garden 
tokenized as [162, 138, 915, 42, 748, 155, 199, 29, 3, 9293]
garden, is to be deleted
In the garden, she gave me a card 
tokenized as [283, 3, 62, 6, 2186, 4333, 42, 748]
She gave me a card in the garden 
tokenized as [0, 2186, 4333, 42, 748, 29, 3, 9293]
Mary hates John 
tokenized as [12219, 20296, 1836]
John hates Mary 
tokenized as [1836, 20296, 12219]
Mary does not like John 
tokenized as [12219, 2724, 206, 473, 1836]
John does not like Mary 
tokenized as [1836, 2724, 206, 473, 12219]


In [203]:
print(x)

                                              tensor        label  0a card0  \
0  [-0.5745196, -0.00013382416, 0.008791878, 0.21...      a card0      1.00   
1  [0.078547575, -0.003191492, -0.016591033, -0.5...  the garden0      0.46   
2  [0.10111215, -0.0076690153, -0.052583955, -0.7...  the garden1      0.39   
3  [0.12289831, -0.008652479, -0.03614599, -0.612...  the garden1      0.38   
4  [-0.5221896, -0.0010482726, -0.027279004, 0.20...      a card2      0.80   
5  [0.054261487, -0.004450589, -0.029107869, -0.6...  the garden2      0.45   
6  [0.0023896925, -0.0038922853, -0.0048231874, 0...  hates John3      0.07   
7  [0.018832594, -0.0017579616, -0.021168415, 0.0...  hates Mary3      0.12   
8  [0.00914142, -0.003637821, -0.018843567, 0.003...   like John4      0.03   
9  [0.031178458, -0.0034231788, -0.10273134, 0.00...   like Mary4      0.10   

   1the garden0  2the garden1  3the garden1  4a card2  5the garden2  \
0          0.46          0.39          0.38      0.80      

In [199]:
def cos(x,y):
    dist = np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
    return dist.round(2)

In [207]:
for i,each in enumerate(x.index):
    label = str(i)+x.loc[each]['label']
    x[label] = x['tensor'].apply(lambda y: cos(y,x.loc[each]['tensor']))
x1=x.drop(['tensor'],axis=1).set_index('label')
x1

,0a card0,1the garden0,2the garden1,3the garden1,4a card2,5the garden2,6hates John3,7hates Mary3,8like John4,9like Mary4
label,,,,,,,,,,
a card0,1.00,0.46,0.39,0.38,0.80,0.45,0.07,0.12,0.03,0.10
the garden0,0.46,1.00,0.91,0.91,0.43,0.96,0.03,0.07,-0.01,0.05
the garden1,0.39,0.91,1.00,0.96,0.38,0.91,0.04,0.08,-0.01,0.04
the garden1,0.38,0.91,0.96,1.00,0.37,0.89,0.02,0.06,-0.00,0.04
a card2,0.80,0.43,0.38,0.37,1.00,0.48,0.03,0.09,-0.01,0.08
the garden2,0.45,0.96,0.91,0.89,0.48,1.00,0.05,0.08,0.00,0.05
hates John3,0.07,0.03,0.04,0.02,0.03,0.05,1.00,0.60,0.83,0.47
hates Mary3,0.12,0.07,0.08,0.06,0.09,0.08,0.60,1.00,0.43,0.82
like John4,0.03,-0.01,-0.01,-0.00,-0.01,0.00,0.83,0.43,1.00,0.54


In [208]:
mask = np.zeros_like(x1)
mask[np.triu_indices_from(mask)] = True

In [209]:
f = plt.figure(figsize = (8.5,8.5)).add_subplot(1,1,1)
ax = sns.heatmap(x1,vmin=-0.1,vmax=1,mask=mask)
# plt.title('all modulated embeddings')
plt.title('all heat')
plt.show()